In [ ]:
import matplotlib.pyplot as plt
!pip install map-boxes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
from pathlib import Path
import random
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.7/dist-packages/map_boxes/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_dir = "../drive/MyDrive/ttrstn/test_dataset_test/test/images"
len(os.listdir(test_dir))

855

In [ ]:
train_dir = "../drive/MyDrive/ttrstn/train_dataset_train/train"
len(os.listdir(f"{train_dir}/images"))

2002

In [ ]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir(f"{train_dir}/labels")]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [ ]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo
!mkdir data_for_yolo/data
!mkdir data_for_yolo/data/images
!mkdir data_for_yolo/data/labels
!mkdir data_for_yolo/data/images/train
!mkdir data_for_yolo/data/labels/train
!mkdir data_for_yolo/data/images/test
!mkdir data_for_yolo/data/labels/test

In [ ]:
def label_to_image_name(s):
    return f"{'_'.join(s.split('_')[:-1])}.jpg".replace("labels", "images")

In [ ]:
# class  lookup
class_lookup = dict(zip(
    ["car", "head", "face", "human", "carplate"],
    range(5)
))
# get names of all images
images_names = [str(p) for p in Path(f"{train_dir}/images").rglob("*.jpg")]
# create a lookup table for merging labels
lookup = {image_name: [] for image_name in images_names}
# go through each image and collect all labels in one place
labels = [str(p) for p in Path(f"{train_dir}/labels").rglob("*.txt")]
log_rate = 100
for i, label in enumerate(labels):
    if i % log_rate == 0:
        print(i, ' / ', len(labels))
    # retrieve the class name of labels
    class_name = label.split('_')[-1].split('..')[0]
    # retrieve the image name
    image_name = label_to_image_name(label)
    # append labeling info to the image
    with open(label, 'r') as f:
        # going through each labeled object
        for line in f:
            # removing the incorrect label
            bbox = ' '.join(line.strip('\n').split(' ')[1:])
            # appending the correct label
            lookup[image_name].append(f"{class_lookup[class_name]} {bbox}")
        # print(lookup[image_name])
    # print(class_name)

In [ ]:
train_images_names, test_images_names = train_test_split(images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

1601 401


In [ ]:
#train
config = {
    "train": train_images_names,
    "test": test_images_names
}
log_rate = 100
for folder, images_names in config.items():
    for i, image_name in enumerate(images_names):
        if i % log_rate == 0:
            print(i, ' / ', len(images_names))
        # move the image
        shutil.copy(image_name, f"data_for_yolo/data/images/{folder}/")
        # create the labeling txt file
        with open(f"data_for_yolo/data/labels/{folder}/{image_name.split('/')[-1].replace('.jpg', '.txt')}", 'w+') as f:
            for obj in lookup[image_name]:
                f.write(f"{obj}\n")

0  /  1601
100  /  1601
200  /  1601
300  /  1601
400  /  1601
500  /  1601
600  /  1601
700  /  1601
800  /  1601
900  /  1601
1000  /  1601
1100  /  1601
1200  /  1601
1300  /  1601
1400  /  1601
1500  /  1601
1600  /  1601
0  /  401
100  /  401
200  /  401
300  /  401
400  /  401


In [ ]:
#валидация 
print(len(os.listdir('data_for_yolo/data/images/test')))
print(len(os.listdir('data_for_yolo/data/images/train')))

401
1601


In [ ]:
!ls ../drive/MyDrive/ttrstn

dataset_5_cls.yaml  Resnet50_Final.pth	 yolov7-e6e_training.pt
dataset_face.yaml   test_dataset_test
exp		    train_dataset_train


In [ ]:
path_to_data = '../drive/MyDrive/ttrstn/dataset_5_cls.yaml'
save_path = '../drive/MyDrive/ttrstn'
pretrained = '../drive/MyDrive/ttrstn/yolov7-e6e_training.pt'

In [ ]:
!python train_aux.py --batch 8 \
                     --cfg cfg/training/yolov7-e6e.yaml \
                     --epochs 200 \
                     --data {path_to_data} \
                     --weights {pretrained} \
                     --device 0 \
                     --image-weights \
                     --cache-images \
                     --project {save_path} \
                     --save_period 100

YOLOR 🚀 v0.1-105-g064c71e torch 1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=True, cfg='cfg/training/yolov7-e6e.yaml', data='../drive/MyDrive/ttrstn/dataset_5_cls.yaml', device='0', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=True, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='../drive/MyDrive/ttrstn', quad=False, rect=False, resume=False, save_dir='../drive/MyDrive/ttrstn/exp', save_period=100, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='../drive/MyDrive/ttrstn/yolov7-e6e_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir ../drive/MyDrive/ttrstn', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, 